In [4]:
# CNN with complete topology (convolutional layer with 32 feature maps and 5x5 filter ->
# convolutional layer with 64 feature maps and 5x5 filter -> fully connected layer -> 
# dropout -> fully connected layer -> softmax)

from __future__ import print_function

import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("../data/raw/MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Fully connected layer 2 (Output layer)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 15000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting ../data/raw/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/raw/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/raw/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/raw/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0 0.1046
100 0.8446
200 0.907
300 0.9301
400 0.9387
500 0.9419
600 0.9552
700 0.9553
800 0.9599
900 0.9647
1000 0.9625
1100 0.9686
1200 0.9705
1300 0.9709
1400 0.9731
1500 0.974
1600 0.9745
1700 0.9757
1800 0.9768
1900 0.9767
2000 0.977
2100 0.9773
2200 0.9788
2300 0.9786
2400 0.9791
2500 0.9799
2600 0.9809
2700 0.9764
2800 0.9812
2900 0.9817
3000 0.9801
3100 0.9832
3200 0.9822
3300 0.9823
3400 0.9842
3500 0.983
3600 0.983
3700 0.9835
3800 0.9835
3900 0.9842
4000 0.9834
4100 0.9823
4200 0.986
4300 0.9852
4400 0.986
4500 0.9848
4600 0.9851
4700 0.9855
4800 0.9859
4900 0.9863
5000 0.9858
5100 0.9861
5200 0.9875
5300 0.9855
5400 0.9875
55

In [18]:
# CNN 1st variation (convolutional layer with 25 feature maps and 5x5 filter -> convolutional
# layer with 50 feature maps and 5x5 filter -> fully connected layer -> dropout ->
# fully connected layer -> softmax)

from __future__ import print_function

import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("../data/raw/MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([5, 5, 1, 25])
b_conv1 = bias_variable([25])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([5, 5, 25, 50])
b_conv2 = bias_variable([50])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 50])

W_fc1 = weight_variable([7 * 7 * 50, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Fully connected layer 2 (Output layer)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 15000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.084
100 0.8233
200 0.8983
300 0.9227
400 0.9357
500 0.9429
600 0.9437
700 0.9516
800 0.9529
900 0.9607
1000 0.9619
1100 0.9621
1200 0.9648
1300 0.9664
1400 0.9684
1500 0.9711
1600 0.9726
1700 0.9736
1800 0.9736
1900 0.9753
2000 0.9757
2100 0.9765
2200 0.9753
2300 0.978
2400 0.9783
2500 0.9785
2600 0.9785
2700 0.9802
2800 0.9795
2900 0.9812
3000 0.9817
3100 0.9814
3200 0.9793
3300 0.9809
3400 0.9818
3500 0.9822
3600 0.9832
3700 0.9814
3800 0.9823
3900 0.9842
4000 0.9844
4100 0.9848
4200 0.9853
4300 0.985
4400 0.9845
4500 0.982
4600 0.9851
4700 0.9846
4800 0.9866
4900 0.9839
5000 0.9849
5100 0.9847
5200 0.9863
5300 0.986
5400 0.9857
5500 0.9851
5600 0.9862
5700 0.9873
5800 0.9864
5900 0.9877
6000 0.986
6100 0.986
6200 0.9878
6300 0.9877
6400 0.9873
6500 0.9878
6600 0.987
6700 0.9873
6800 0.987

In [19]:
# CNN 2nd variation (convolutional layer with 25 feature maps and 3x3 filter -> convolutional
# layer with 50 feature maps and 4x4 filter -> fully connected layer -> dropout -> fully
# connected layer -> softmax)

from __future__ import print_function

import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("../data/raw/MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([3, 3, 1, 25])
b_conv1 = bias_variable([25])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([4, 4, 25, 50])
b_conv2 = bias_variable([50])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7* 50])

W_fc1 = weight_variable([7 * 7 * 50, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Fully connected layer 2 (Output layer)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 15000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.0785
100 0.7933
200 0.8811
300 0.9126
400 0.9228
500 0.9331
600 0.938
700 0.944
800 0.9443
900 0.9499
1000 0.9551
1100 0.9508
1200 0.9589
1300 0.9603
1400 0.9614
1500 0.9649
1600 0.9684
1700 0.9657
1800 0.9681
1900 0.9709
2000 0.9696
2100 0.9721
2200 0.9722
2300 0.9715
2400 0.9764
2500 0.9732
2600 0.9759
2700 0.9768
2800 0.9774
2900 0.9792
3000 0.9795
3100 0.9795
3200 0.9778
3300 0.9803
3400 0.9793
3500 0.9809
3600 0.9796
3700 0.9809
3800 0.9814
3900 0.9819
4000 0.9817
4100 0.9832
4200 0.9838
4300 0.9822
4400 0.9819
4500 0.983
4600 0.9841
4700 0.9825
4800 0.9852
4900 0.9836
5000 0.985
5100 0.9854
5200 0.9849
5300 0.9856
5400 0.9851
5500 0.9847
5600 0.9859
5700 0.9862
5800 0.9865
5900 0.9868
6000 0.986
6100 0.9869
6200 0.9864
6300 0.9865
6400 0.9865
6500 0.9863
6600 0.9869
6700 0.9878
6800 0.

In [20]:
# CNN 3rd variation (convolutional layer with 25 feature maps and 3x3 filter -> convolutional
# layer with 50 feature maps and 4x4 filter -> fully connected layer -> softmax)

from __future__ import print_function

import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("../data/raw/MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([3, 3, 1, 25])
b_conv1 = bias_variable([25])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([4, 4, 25, 50])
b_conv2 = bias_variable([50])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 50])

W_fc1 = weight_variable([7 * 7 * 50, 10])
b_fc1 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
# keep_prob  = tf.placeholder(tf.float32)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Fully connected layer 2 (Output layer)
# W_fc2 = weight_variable([1024, 10])
# b_fc2 = bias_variable([10])

# y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 15000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.1151
100 0.4743
200 0.7299
300 0.7924
400 0.8525
500 0.8674
600 0.8869
700 0.8974
800 0.9033
900 0.9112
1000 0.9114
1100 0.9175
1200 0.9206
1300 0.9254
1400 0.927
1500 0.9302
1600 0.9329
1700 0.932
1800 0.9382
1900 0.9394
2000 0.9431
2100 0.9425
2200 0.9399
2300 0.9475
2400 0.9481
2500 0.9453
2600 0.9514
2700 0.9525
2800 0.9519
2900 0.9547
3000 0.9568
3100 0.9557
3200 0.9595
3300 0.9579
3400 0.9603
3500 0.9595
3600 0.9589
3700 0.9615
3800 0.9623
3900 0.9634
4000 0.9644
4100 0.9653
4200 0.967
4300 0.9658
4400 0.968
4500 0.9674
4600 0.9681
4700 0.9702
4800 0.9696
4900 0.9712
5000 0.9719
5100 0.9711
5200 0.9735
5300 0.971
5400 0.9706
5500 0.9716
5600 0.9717
5700 0.9742
5800 0.9732
5900 0.9743
6000 0.9751
6100 0.9752
6200 0.9752
6300 0.9767
6400 0.9759
6500 0.9777
6600 0.9762
6700 0.978
6800 0.9

In [2]:
# CNN 4th variation (convolutional layer with 25 feature maps and 3x3 filter -> convolutional
# layer with 50 feature maps and 4x4 filter -> dropout -> fully connected layer -> softmax)

from __future__ import print_function

import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("../data/raw/MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([3, 3, 1, 25])
b_conv1 = bias_variable([25])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([4, 4, 25, 50])
b_conv2 = bias_variable([50])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 50])

# W_fc1 = weight_variable([7 * 7 * 50, 1024])
# b_fc1 = bias_variable([1024])

# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_pool2_flat, keep_prob)

# Fully connected layer 2 (Output layer)
W_fc2 = weight_variable([7 * 7 * 50, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_pool2_flat, W_fc2) + b_fc2, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 15000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting ../data/raw/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/raw/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/raw/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/raw/MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.084
100 0.4402
200 0.6688
300 0.7919
400 0.8273
500 0.8619
600 0.8679
700 0.8859
800 0.8987
900 0.9036
1000 0.9074
1100 0.9095
1200 0.9166
1300 0.9202
1400 0.924
1500 0.9276
1600 0.9315
1700 0.9302
1800 0.9344
1900 0.9355
2000 0.9372
2100 0.9401
2200 0.9428
2300 0.9424
2400 0.945
2500 0.9476
2600 0.9475
2700 0.9496
2800 0.9508
2900 0.9508
3000 0.9546
3100 0.9554
3200 0.9553
3300 0.9568
3400 0.9587
3500 0.9572
3600 0.9596
3700 0.9604
3800 0.9589
3900 0.9601
4000 0.9629
4100 0.9632
4200 0.9667
4300 0.9662
4400 0.9648
4500 0.9669
4600 0.9675
4700 0.9709
4800 0.9694
4900 0.9703
5000 0.9702
5100 0.9692
5200 0.9726
5300 0.9705
5400 0.9724
5500 0.9736
5600 0.9725
5700 0.9727
5800 0.9738
5900 0.9746
6000 0.9751
6100 0.9752
6200 0.9752
6300 0.9751
6400 